# Train a neural network for binary volumetric brain extraction

In this notebook, we will use the `nobrainer` python API to train two models for brain extraction. Brain extraction is a common step in processing neuroimaging data. It is a voxel-wise, binary classification task, where each voxel is classified as brain or not brain.

In the following cells, we will:

1. Get sample T1-weighted MR scans as features and FreeSurfer segmentations as labels.
2. Convert the data to TFRecords format for use with neural networks.
3. Create two `Datasets` of features and labels, one for training, one for evaluation.
4. Instantiate a 3D convolutional neural network model for image segmentation called U-Net.
5. Train on part of the data and evaluate on the rest of the data.
6. Predict a brain mask using the trained model.
7. Save the model to disk for future prediction and/or training.
8. Load the model back from disk and show that brain extraction works as before saving.
9. Demonstrate the same workflow using a different model for brain extraction called MeshNet.


## Google Colaboratory

If you are using Colab, please switch your runtime to GPU. To do this, select `Runtime > Change runtime type` in the top menu. Then select GPU under `Hardware accelerator`. A GPU greatly speeds up training.


# Install and setup `nobrainer`

In [ ]:
!pip install nobrainer nilearn

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import nobrainer

# Get sample features and labels

We use 9 pairs of volumes for training and 1 pair of volumes for evaluation. Many more volumes would be required to train a model for any useful purpose.

In [ ]:
csv_path = nobrainer.utils.get_data()
filepaths = nobrainer.io.read_csv(csv_path)

# Convert medical images to TFRecords

In [ ]:
from nobrainer.dataset import Dataset

In [ ]:
n_epochs = 2
DT = Dataset(n_classes=1,
             batch_size=2,
             block_shape=(128, 128, 128),
             n_epochs=n_epochs)

dataset_train, dataset_eval = DT.from_files(paths=filepaths,
                                            eval_size=0.1,
                                            tfrecdir="data/binseg",
                                            shard_size=3,
                                            augment=None,
                                            shuffle_buffer_size=10,
                                            num_parallel_calls=None)

# U-Net model for brain mask extraction

## Construct the model
Here we'll train `nobrainer`'s implementation of the U-Net model for biomedical image segmentation, based on https://arxiv.org/abs/1606.06650.

`nobrainer` provides several other segmentation models that could be used instead of `unet`. Another example is provided at the bottom of this guide, and for a complete list, see [this list](https://github.com/neuronets/nobrainer#models).

Note that a useful segmentation model would need to be trained on *many* more examples than the 10 we are using here for demonstration.

In [ ]:
from nobrainer.processing.segmentation import Segmentation
from nobrainer.models import unet

bem = Segmentation(unet, model_args=dict(batchnorm=True))

## Train the model on the example data
A summary of the model layers is printed before training starts.

Note that the loss function after training is very high, and the dice coefficient (a measure of the accuracy of the model) is very low, indicating that the model is not doing a good job of binary segmentation. This is expected, as this is a toy problem to demonstrate the API. During successful training of a more practical model, you would see the loss drop and the dice rise as training progressed.

In [ ]:
history = bem.fit(dataset_train=dataset_train,
                  dataset_validate=dataset_eval,
                  epochs=n_epochs,
                  multi_gpu=True,
                  )

## Use the trained model to predict a binary brain mask
The segmentation is bad, but that isn't surprising given the small dataset and short training.

In [ ]:
import matplotlib.pyplot as plt
from nilearn import plotting
from nobrainer.volume import standardize

image_path = filepaths[0][0]
out = bem.predict(image_path, normalizer=standardize)
out.shape

fig = plt.figure(figsize=(12, 6))
plotting.plot_roi(out, bg_img=image_path, cut_coords=(0, 10, -21), alpha=0.4, vmin=0, vmax=5, figure=fig)

## Save the trained model

In [ ]:
bem.save("data/unet-brainmask-toy")

## Load the model from disk

In [ ]:
bem = Segmentation.load("data/unet-brainmask-toy")

## Predict a brain mask from the loaded model
The brain mask is identical to that predicted before saving.

In [ ]:
image_path = filepaths[0][0]
out = bem.predict(image_path, normalizer=standardize)
out.shape

fig = plt.figure(figsize=(12, 6))
plotting.plot_roi(out, bg_img=image_path, cut_coords=(0, 10, -21), alpha=0.4, vmin=0, vmax=5, figure=fig)

# Another model for brain mask extraction

## Construct a MeshNet model
Here we'll train `nobrainer`'s implementation of the MeshNet model for biomedical image segmentation, based on https://arxiv.org/abs/1612.00940).

In [ ]:
from nobrainer.models import meshnet
bem = Segmentation(meshnet, model_args=dict(filters=25))

## Train the MeshNet model on the example data
A summary of the model layers is printed before training starts.

In [ ]:
history = bem.fit(dataset_train=dataset_train,
                  dataset_validate=dataset_eval,
                  epochs=n_epochs,
                  multi_gpu=True,
                  )

## Use the trained MeshNet model to predict a binary brain mask

In [ ]:
import matplotlib.pyplot as plt
from nilearn import plotting
from nobrainer.volume import standardize

image_path = filepaths[0][0]
out = bem.predict(image_path, normalizer=standardize)
out.shape

fig = plt.figure(figsize=(12, 6))
plotting.plot_roi(out, bg_img=image_path, cut_coords=(0, 10, -21), alpha=0.4, vmin=0, vmax=5, figure=fig)

## Save the MeshNet model

In [ ]:
bem.save("data/meshnet-brainmask-toy")